In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv
from datetime import datetime
import schedule

In [ ]:
# Função para coletar dados
def coletar_dados():
    driver = webdriver.Chrome()
    url = 'https://seminovos.localiza.com/carros/pr-curitiba?cidade=pr-curitiba'
    driver.get(url)

    # Aguardar até que alguns elementos estejam presentes (pode ser necessário ajustar o tempo)
    driver.implicitly_wait(10)

    # Encontrar elementos com a classe 'title-car' e 'subtitle-car-primary'
    elements_with_title_car_class = driver.find_elements(By.CLASS_NAME, 'title-car')
    elements_with_subtitle_car_class = driver.find_elements(By.CLASS_NAME, 'subtitle-car-primary')
    elements_with_year_class = driver.find_elements(By.CLASS_NAME, 'text-km')
    elements_with_km_class = driver.find_elements(By.CLASS_NAME, 'text-milage')
    elements_with_gearbox_class = driver.find_elements(By.CLASS_NAME, 'text-gearbox')
    elements_with_price_car_class = driver.find_elements(By.CLASS_NAME, 'text-price')

    # Cria lista para armazenar as informações dos carros
    car_info_list = []

    # Função para rolagem infinita
    def scroll_down():
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)  # Aguardar um curto período após rolar para baixo (ajuste conforme necessário)

    # rolagem até que todos os produtos sejam carregados
    while len(elements_with_title_car_class) > 0 :
        scroll_down()
        elements_with_title_car_class = driver.find_elements(By.CLASS_NAME, 'title-car')
        elements_with_subtitle_car_class = driver.find_elements(By.CLASS_NAME, 'subtitle-car-primary')
        elements_with_year_class = driver.find_elements(By.CLASS_NAME, 'text-km')
        elements_with_km_class = driver.find_elements(By.CLASS_NAME, 'text-milage')
        elements_with_gearbox_class = driver.find_elements(By.CLASS_NAME, 'text-gearbox')
        elements_with_price_car_class = driver.find_elements(By.CLASS_NAME, 'text-price')
        
    # Obter a data atual:
    data_atual = datetime.now().strftime('%d/%m/%Y')

    # Iterar sobre os elementos encontrados e adicionar a data
    for i in range(len(elements_with_title_car_class)):
        marca = elements_with_title_car_class[i].text.strip()
        modelo = elements_with_subtitle_car_class[i].text.strip()
        ano = elements_with_year_class[i].text.strip()
        km = elements_with_km_class[i].text.strip()
        cambio = elements_with_gearbox_class[i].text.strip()
        preco = elements_with_price_car_class[i].text.strip()

        # Adicionar as informações à lista com a data atual
        car_info_list.append({'data': data_atual, 'marca': marca, 'modelo': modelo, 'ano': ano, 'km': km, 'cambio' : cambio, 'preco': preco})

    # Salvar as informações CSV:
    csv_filename = 'car_info_list.csv'
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['data', 'marca', 'modelo', 'ano', 'km', 'cambio', 'preco']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for car_info in car_info_list:
            writer.writerow({'data': car_info['data'], 'marca': car_info['marca'], 'modelo': car_info['modelo'], 'ano': car_info['ano'], 'km': car_info['km'], 'cambio': car_info['cambio'], 'preco': car_info['preco']})

    # Imprimir informações dos carros
    print(f"Informações dos carros foram exportadas para '{csv_filename}'.")

    # Fechar o WebDriver
    driver.quit()

# Agendar função coletar_dados para as 18:00 todos os dias:
schedule.every().day.at("18:00").do(coletar_dados)

# Loop para continuar verificando se há tarefas agendadas para executar
while True:
    schedule.run_pending()
    time.sleep(60)  # Aguardar 60 segundos antes de verificar novamente
